# Automated segmentation of thorax CT scans using neural networks

### CS 180 MP 2
### Rianna Patricia Cruz

## Introduction 
The identification of tumors and abnormal masses in medical imaging scans is one of the most important steps in the treatment of such abnormal conditions. Such a essential step in the treatment process is often tedious and time-consuming, as it requires a medical professional to manually identify such masses igiven some medical images such as CT or MRI scans. Though such identification process might be straightforward for a medical professional, the time it takes for such scans to be manually and individually processed and analyzed by a medical professional causes delays which might be critical to a patient's treatment.

The objective of this study was to automatically segment four organs-at-risk (OAR): the heart, the aorta, the trachea, and the esophagus


In [2]:
import datasets

In [3]:
print("[INFO] loading scan attributes...")
scans, masks = datasets.load_scans()

[INFO] loading scan attributes...


In [4]:
scans.shape
masks.shape

(7420, 512, 512)

In [5]:
from sklearn.model_selection import train_test_split
# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
print("main: splitting data...")
split = train_test_split(scans, masks, test_size=0.25, random_state=42)
(X_train, X_test, y_train, y_test) = split
print("main: done split")

main: splitting data...
main: done split


In [7]:
X_train_reshape = X_train.reshape(X_train.shape[0],-1)
y_train_reshape = y_train.reshape(y_train.shape[0],-1)
X_test_reshape = X_test.reshape(X_test.shape[0],-1)
y_test_reshape = y_test.reshape(y_test.shape[0],-1)

print(X_train_reshape.shape)
print(y_train_reshape.shape)
print(X_test_reshape.shape)
print(y_test_reshape.shape)

(5565, 262144)
(5565, 262144)
(1855, 262144)
(1855, 262144)


In [ ]:
# https://www.springboard.com/blog/beginners-guide-neural-network-in-python-scikit-learn-0-18/

In [ ]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(hidden_layer_sizes=(13,13,13),max_iter=50)

mlp.fit(X_train_reshape,y_train_reshape)


In [ ]:
MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(13, 13, 13), learning_rate='constant',
       learning_rate_init=0.001, max_iter=500, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)


In [143]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

print(X_train.shape[1:])
print(y_train.shape[1:])
print(X_test.shape[1:])
print(y_test.shape[1:])

(5565, 512, 512)
(5565, 512, 512)
(1855, 512, 512)
(1855, 512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)


In [144]:
# transpose for the sake of splitting training and test data
X_train.shape
X_test.shape

(1855, 512, 512)

In [147]:
from sknn.mlp import Regressor, Layer
import sklearn.cross_validation

nn = Regressor(
    layers=[
        Layer("Rectifier", units=100),
        Layer("Linear")],
    learning_rate=0.02,
    n_iter=10)
nn.fit(X_train, y_train)

ModuleNotFoundError: No module named 'sklearn.cross_validation'

In [38]:
import numpy as np 
X_train = np.expand_dims(X_train, axis=4)
y_train = np.expand_dims(y_train, axis=4)
X_test = np.expand_dims(X_test, axis=4)
y_test = np.expand_dims(y_test, axis=4)

/Users/rc/Documents/Acads/1819/180/MP2/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  
/Users/rc/Documents/Acads/1819/180/MP2/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/rc/Documents/Acads/1819/180/MP2/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  after removing the cwd from sys.path.
/Users/rc/Documents/Acads/1819/180/MP2/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  """


In [40]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

print(X_train.shape[1:])
print(y_train.shape[1:])
print(X_test.shape[1:])
print(y_test.shape[1:])

(5565, 512, 512, 1)
(5565, 512, 512, 1)
(1855, 512, 512, 1)
(1855, 512, 512, 1)
(512, 512, 1)
(512, 512, 1)
(512, 512, 1)
(512, 512, 1)


In [138]:
# https://adventuresinmachinelearning.com/keras-tutorial-cnn-11-lines/
from keras.models import Sequential
from keras.layers import Conv3D, Dense, Flatten, MaxPooling3D, Dropout
from keras.layers import TimeDistributed, LSTM
from keras.layers import AveragePooling1D, GlobalAveragePooling1D
from keras.layers import Conv2D, Dense, Flatten, MaxPooling2D
from keras.layers import Input

from keras.losses import sparse_categorical_crossentropy
from keras.optimizers import RMSprop

num_classes = 4

input_shape = (512, 512, 1)

model = Sequential()
model.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1),
                 activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(64, (5, 5), 
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(4, activation='softmax'))

'''
model = Sequential()
model.add(LSTM(24, input_shape=(1200, 19), return_sequences=True, implementation=2))
model.add(TimeDistributed(Dense(1)))
model.add(AveragePooling1D())

model.add(GlobalAveragePooling1D())

model.add(Dense(2, activation='softmax'))
model.compile(loss=sparse_categorical_crossentropy, optimizer=RMSprop(lr=.01))
model.fit(X_train, y_train, epochs=100, batch_size=6000, verbose=1, validation_data=(X_test, y_test))

input_shape=(512, 512, 1)
model = Sequential()
model.add(Dense(64, input_dim=20, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

'''
print(model.summary())

ValueError: Input 0 is incompatible with layer conv2d_41: expected ndim=4, found ndim=3

In [139]:

X_train_validate = X_train.reshape(-1, 1)
print(X_train_validate.ndim)

y_train_validate = y_train.reshape(-1, 1)
print(y_train_validate.ndim)

X_test_validate = X_test.reshape(-1, 1)
print(X_test_validate.ndim)

y_test_validate = y_test.reshape(-1,1)
print(y_test_validate.ndim)

2
2
2
2


In [137]:
model.compile(loss=sparse_categorical_crossentropy, optimizer=RMSprop(lr=.01))
model.fit(X_train, y_train, epochs=100, batch_size=6000, verbose=1, validation_data=(X_test, y_test))


ValueError: Error when checking target: expected dense_46 to have 2 dimensions, but got array with shape (5565, 512, 512, 1)

In [ ]:
from keras.losses import sparse_categorical_crossentropy
from keras.optimizers import SGD

model.compile(loss=sparse_categorical_crossentropy,
              optimizer=SGD(lr=0.01),
              metrics=['accuracy'])

In [59]:
from keras.callbacks import Callback
class AccuracyHistory(Callback):
    def on_train_begin(self, logs={}):
        self.acc = []

    def on_epoch_end(self, batch, logs={}):
        self.acc.append(logs.get('acc'))

In [60]:
'''
X_train_try = X_train.reshape(-1, 512, 512, 1)
X_test_try = X_test.reshape(-1, 512, 512, 1)
y_train_try = y_train.reshape(-1, 512, 512, 1)
y_test_try = y_test.reshape(-1, 512, 512, 1)
'''


'\nX_train_try = X_train.reshape(-1, 512, 512, 1)\nX_test_try = X_test.reshape(-1, 512, 512, 1)\ny_train_try = y_train.reshape(-1, 512, 512, 1)\ny_test_try = y_test.reshape(-1, 512, 512, 1)\n'

In [61]:
print(X_train_try.shape)
print(y_train_try.shape)
print(X_test_try.shape)
print(y_test_try.shape)

print(X_train_try.shape[1:])
print(y_train_try.shape[1:])
print(X_test_try.shape[1:])
print(y_test_try.shape[1:])

NameError: name 'X_train_try' is not defined

In [62]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [54]:
batch_size = 50
epochs = 20


model.fit(X_train, y_train,
          epochs=20,
          batch_size=128)
'''
history = AccuracyHistory()

model.fit(X_train_try, y_train_try,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test_try, y_test_try),
          callbacks=[history])
'''

ValueError: Error when checking target: expected dense_6 to have 2 dimensions, but got array with shape (5565, 512, 512, 1)